In [1]:
%pip install pytorch-lightning pytorch-forecasting pandas matplotlib seaborn scikit-learn tqdm fastparquet optuna --quiet

import pandas as pd
import numpy as np

import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline, GroupNormalizer
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

Note: you may need to restart the kernel to use updated packages.


/Users/lucareggiani/Documents/GitHub/techHub/submodules/Multimodal-Anomaly-Detection-for-Marketing/.venv/lib/python3.12/site-packages/pytorch_forecasting/models/base_model.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_parquet("./data/processed/long_table_multimodal_labelencoded_timeidx.parquet")
print(df.shape)
df.head(20)

(1282604, 232)


,Unnamed: 0,external_code,retail,season,category,color,image_path,fabric,release_date,restock,...,devore_trend,hron_trend,ottoman_trend,fluid_trend,flamed_trend,fluid polyviscous_trend,shiny jersey_trend,goose_trend,time_idx,target
0,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,19.0,18.0,71.0,84.0,44.0,-1.0,0.0,51.0,1,3.0
1,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,20.0,15.0,64.0,78.0,37.0,-1.0,0.0,55.0,2,3.0
2,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,12.0,17.0,56.0,78.0,48.0,-1.0,35.0,58.0,3,1.0
3,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,27.0,20.0,77.0,67.0,74.0,-1.0,0.0,55.0,4,4.0
4,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,27.0,22.0,78.0,76.0,38.0,-1.0,0.0,45.0,5,3.0
5,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,36.0,9.0,76.0,81.0,32.0,-1.0,32.0,42.0,6,0.0
6,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,34.0,13.0,74.0,79.0,72.0,-1.0,0.0,40.0,7,1.0
7,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,27.0,24.0,76.0,84.0,43.0,-1.0,0.0,37.0,8,0.0
8,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,24.0,20.0,76.0,83.0,36.0,-1.0,32.0,34.0,9,0.0
9,133,1,3,SS17,long sleeve,violet,PE17/00001.png,acrylic,2016-12-05,20,...,29.0,18.0,71.0,83.0,68.0,-1.0,0.0,34.0,10,0.0


In [18]:
df['date_x'].unique()


<DatetimeArray>
['2016-12-05 00:00:00', '2016-12-12 00:00:00',                 'NaT',
 '2017-01-02 00:00:00', '2017-01-09 00:00:00', '2017-02-06 00:00:00',
 '2017-03-06 00:00:00', '2017-08-07 00:00:00', '2017-09-04 00:00:00',
 '2017-09-11 00:00:00', '2017-10-02 00:00:00', '2017-07-10 00:00:00',
 '2017-04-03 00:00:00', '2017-04-10 00:00:00', '2018-02-05 00:00:00',
 '2018-02-12 00:00:00', '2018-03-05 00:00:00', '2018-03-12 00:00:00',
 '2018-04-02 00:00:00', '2018-04-09 00:00:00', '2017-10-09 00:00:00',
 '2017-07-03 00:00:00', '2017-06-05 00:00:00', '2017-06-12 00:00:00',
 '2017-05-08 00:00:00', '2017-05-01 00:00:00', '2018-01-08 00:00:00',
 '2017-11-06 00:00:00', '2018-08-06 00:00:00', '2018-09-03 00:00:00',
 '2018-09-10 00:00:00', '2018-10-01 00:00:00', '2018-10-08 00:00:00',
 '2017-12-11 00:00:00', '2018-01-01 00:00:00', '2017-12-04 00:00:00',
 '2018-11-05 00:00:00', '2018-05-07 00:00:00', '2018-06-04 00:00:00',
 '2018-07-02 00:00:00', '2018-07-09 00:00:00', '2018-06-11 00:00:00',
 '20

In [11]:

# Ottieni l'elenco delle colonne
columns = df.columns.tolist()

# Stampa l'elenco delle colonne
print(columns)

['Unnamed: 0', 'external_code', 'retail', 'season', 'category', 'color', 'image_path', 'fabric', 'release_date', 'restock', 'week', 'sales', 'cat_idx', 'color_idx', 'fabric_idx', 'img_pca_0', 'img_pca_1', 'img_pca_2', 'img_pca_3', 'img_pca_4', 'img_pca_5', 'img_pca_6', 'img_pca_7', 'img_pca_8', 'img_pca_9', 'img_pca_10', 'img_pca_11', 'img_pca_12', 'img_pca_13', 'img_pca_14', 'img_pca_15', 'img_pca_16', 'img_pca_17', 'img_pca_18', 'img_pca_19', 'img_pca_20', 'img_pca_21', 'img_pca_22', 'img_pca_23', 'img_pca_24', 'img_pca_25', 'img_pca_26', 'img_pca_27', 'img_pca_28', 'img_pca_29', 'img_pca_30', 'img_pca_31', 'img_pca_32', 'img_pca_33', 'img_pca_34', 'img_pca_35', 'img_pca_36', 'img_pca_37', 'img_pca_38', 'img_pca_39', 'img_pca_40', 'img_pca_41', 'img_pca_42', 'img_pca_43', 'img_pca_44', 'img_pca_45', 'img_pca_46', 'img_pca_47', 'img_pca_48', 'img_pca_49', 'img_pca_50', 'img_pca_51', 'img_pca_52', 'img_pca_53', 'img_pca_54', 'img_pca_55', 'img_pca_56', 'img_pca_57', 'img_pca_58', 'img_

In [ ]:
max_prediction_length = 2    # orizzonte forecast
max_encoder_length = 8      # storia di input

# Split: emula lo scenario rolling di forecasting, lasciando le ultime settimane come validation/test
cutoff_val = df["time_idx"].max() - max_prediction_length - 2   # validation: penultimo blocco
cutoff_test = df["time_idx"].max() - max_prediction_length      # test: ultimo blocco

df_train = df[df.time_idx <= cutoff_val]
df_val = df[(df.time_idx > cutoff_val) & (df.time_idx <= cutoff_test)]
df_test = df[df.time_idx > cutoff_test]

print(f"Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

Train: 1068500, Val: 213700, Test: 404


In [4]:
# Statiche: fisse PER SERIE (es: prodotto, negozio, categoria, embedding PCA, ecc.)
static_categoricals = ["external_code", "retail", "category", "color", "fabric"]
static_reals = []

# Dinamiche note in anticipo (passato/futuro): week, price, discount, restock, embed, meteo, trend, ecc.
img_pca_cols = [c for c in df.columns if c.startswith("img_pca_")]
meteo_cols = [c for c in df.columns if c in ['t_avg', 'rain', 'humidity']]    # personalizza!
trend_cols = [c for c in df.columns if "_trend" in c]

time_varying_known_categoricals = ["week", "season"]                         # se encode o non encode "season"
time_varying_known_reals = ["time_idx", "price", "discount", "restock_qty"] + img_pca_cols + meteo_cols + trend_cols

# Dinamiche sconosciute (solo passato): target
time_varying_unknown_reals = ["target"]

print("Static PCA cols:", img_pca_cols)

Static PCA cols: ['img_pca_0', 'img_pca_1', 'img_pca_2', 'img_pca_3', 'img_pca_4', 'img_pca_5', 'img_pca_6', 'img_pca_7', 'img_pca_8', 'img_pca_9', 'img_pca_10', 'img_pca_11', 'img_pca_12', 'img_pca_13', 'img_pca_14', 'img_pca_15', 'img_pca_16', 'img_pca_17', 'img_pca_18', 'img_pca_19', 'img_pca_20', 'img_pca_21', 'img_pca_22', 'img_pca_23', 'img_pca_24', 'img_pca_25', 'img_pca_26', 'img_pca_27', 'img_pca_28', 'img_pca_29', 'img_pca_30', 'img_pca_31', 'img_pca_32', 'img_pca_33', 'img_pca_34', 'img_pca_35', 'img_pca_36', 'img_pca_37', 'img_pca_38', 'img_pca_39', 'img_pca_40', 'img_pca_41', 'img_pca_42', 'img_pca_43', 'img_pca_44', 'img_pca_45', 'img_pca_46', 'img_pca_47', 'img_pca_48', 'img_pca_49', 'img_pca_50', 'img_pca_51', 'img_pca_52', 'img_pca_53', 'img_pca_54', 'img_pca_55', 'img_pca_56', 'img_pca_57', 'img_pca_58', 'img_pca_59', 'img_pca_60', 'img_pca_61', 'img_pca_62', 'img_pca_63', 'img_pca_64', 'img_pca_65', 'img_pca_66', 'img_pca_67', 'img_pca_68', 'img_pca_69', 'img_pca_70'

In [7]:
# Controllo presenza del valore '10' (sia stringa sia numero)
valore_da_cercare = '10'
risultato = []

for col in df_val.columns:
    col_values = df_val[col].astype(str)
    if (col_values == valore_da_cercare).any():
        risultato.append(col)
        
print("Colonne contenenti il valore '10':", risultato)

Colonne contenenti il valore '10': ['Unnamed: 0', 'external_code', 'retail', 'restock', 'week', 'cat_idx', 'fabric_idx']


In [9]:
# Controllo presenza del valore '10' (sia stringa sia numero)
valore_da_cercare = '10'
risultato = []

for col in df_test.columns:
    col_values = df_test[col].astype(str)
    if (col_values == valore_da_cercare).any():
        risultato.append(col)
        
print("Colonne contenenti il valore '10':", risultato)

Colonne contenenti il valore '10': ['retail', 'restock', 'week', 'cat_idx']


In [10]:
# Controllo presenza del valore '10' (sia stringa sia numero)
valore_da_cercare = '10'
risultato = []

for col in df_train.columns:
    col_values = df_train[col].astype(str)
    if (col_values == valore_da_cercare).any():
        risultato.append(col)
        
print("Colonne contenenti il valore '10':", risultato)

Colonne contenenti il valore '10': ['Unnamed: 0', 'external_code', 'retail', 'restock', 'cat_idx', 'fabric_idx', 'time_idx']


In [46]:
for col in static_categoricals:
    for d in [df_train, df_val, df_test]:
        d[col] = d[col].astype(str)

/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/4170431190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[col] = d[col].astype(str)
/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/4170431190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[col] = d[col].astype(str)
/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/4170431190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [47]:
print(df_train.groupby(['external_code','retail']).size().describe())


count    106850.0
mean         10.0
std           0.0
min          10.0
25%          10.0
50%          10.0
75%          10.0
max          10.0
dtype: float64


In [48]:
for d in [df_train, df_val, df_test]:
    d["discount_was_missing"] = d["discount"].isna().astype(int)
    d["discount"] = d["discount"].fillna(0)

/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/2986294004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["discount_was_missing"] = d["discount"].isna().astype(int)
/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/2986294004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["discount"] = d["discount"].fillna(0)
/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/2986294004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [49]:
for trend_col in trend_cols:
    for d in [df_train, df_val, df_test]:
        d[f"{trend_col}_was_missing"] = d[trend_col].isna().astype(int)
        d[trend_col] = d[trend_col].fillna(0)

/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/292431157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[f"{trend_col}_was_missing"] = d[trend_col].isna().astype(int)
/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/292431157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[trend_col] = d[trend_col].fillna(0)
/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_74031/292431157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

Dataset PyTorch Forecasting

In [50]:
# Training set
training_dataset = TimeSeriesDataSet(
    df_train,
    time_idx="time_idx",
    target="target",
    group_ids=["external_code", "retail"],
    static_categoricals=static_categoricals,
    static_reals=static_reals,
    time_varying_known_categoricals=time_varying_known_categoricals,
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=time_varying_unknown_reals,
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    target_normalizer=GroupNormalizer(groups=["external_code", "retail"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# Validation & test
validation_dataset = TimeSeriesDataSet.from_dataset(training_dataset, df_val, predict=True, stop_randomization=True)
test_dataset = TimeSeriesDataSet.from_dataset(training_dataset, df_test, predict=True, stop_randomization=True)

KeyError: "Unknown category '10' encountered. Set `add_nan=True` to allow unknown categories"

Definizione modello Temporal Fusion Transformer

In [ ]:
tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=1e-3,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,                  # quantiles: [0.1, 0.2, ..., 0.9]
    loss=QuantileLoss(),
    reduce_on_plateau_patience=4,
)
print(f"Model params: {tft.size()}")